In [9]:
import math 
import numpy as np
import random
import cv2

In [10]:
def distanceDroite3D(droite,pts):
    
    droiteD=droite[0]-droite[1]
    droiteD=np.int64(droiteD)

    normeDroite=math.sqrt(droiteD[0]**2 + droiteD[1]**2 + droiteD[2]**2)
    AP=pts-droite[0]
    
    cross=np.cross(AP,droiteD)
    cross=np.int64(cross)

    normecross=math.sqrt(cross[0]**2 + cross[1]**2 + cross[2]**2)

    if normeDroite<0.000000000001:
        return 2000
    
    return normecross/normeDroite


def distanceDroite(droite,pts):
    a=(droite[1][1]-droite[0][1])/(droite[1][0]-droite[0][0])
    b=droite[1][1]-a*droite[1][0]
    return abs(a*pts[0]-pts[1]+b)/math.sqrt(a**2+1)

def distanceCoude(droite,pts,type):

    a0=droite[0][0]
    a1=droite[0][1]

    b0=droite[1][0]
    b1=droite[1][1]

    m0=pts[0]
    m1=pts[1]

    X= ( (b0-a0)*(m0-b0)+(b1-a1)*(m1-b1) )/( (b0-a0)**2+(b1-a1)**2 )

    x=b0+(b0-a0)*X
    y=b1+(b1-a1)*X

    D=1000
    coeff=50

    if type==1 and ((a0<b0 and x<b0+coeff) or (a0>b0 and x>b0-coeff)) and ((a1<b1 and y<b1+coeff) or (a1>b1 and y>b1-coeff)):
        D= distanceDroite(droite,pts)

    if type==2 and ((a0<b0 and x>b0-coeff) or (a0>b0 and x<b0+coeff)) and ((a1<b1 and y>b1-coeff) or (a1>b1 and y<b1+coeff)):
        D= distanceDroite(droite,pts)


    #if ( (a0>b0 and x>b0-coeff and x<a0+coeff) or (a0<b0 and x<b0+coeff and x>a0-coeff) ) and ( (a1>b1 and x>b1-coeff and x<a1+coeff) or (a1<b1 and x<b1+coeff and x>a1-coeff) ) :
        #D= distanceDroite(droite,pts)

    return D

In [11]:


def RANSAC(pts,N=500,distanceMin=20,lastPosition=None,D3=False):
    best_score=0
    best_model=None
    best_droite=None
    
    for k in range(N):
        score=0
        
        start=random.randint(0, pts.shape[0]-1)
        end=random.randint(pts.shape[0]//2, pts.shape[0]-1)
        
        droite=[pts[start][:],pts[end][:]]
        
        model=np.array((pts[start],pts[end]))

        firstLoop=True
        for P in pts:
            if D3:
                D=distanceDroite3D(droite,P)
            else:
                D=distanceDroite(droite,P)

            """if firstLoop and lastPosition is not None:
                Dlast=np.linalg.norm(P[:2]-lastPosition[:2])
                if firstLoop  and D<distanceMin and Dlast>distanceMin*15:
                    break"""

            if D<distanceMin:
                """if firstLoop:
                    score+=2"""
                score+=1
                model=np.vstack((model,P))
                
            firstLoop=False

        if score>pts.shape[0]//2 and score>best_score :
            best_droite=droite
            best_model=model
            best_score=score

        model=None
        
    if best_model is None:
        return None,None
    else:
        return best_model[2:],best_droite
    
def RANSACcoude(pts,N=500,distanceMin=20,lastPosition=None):
    best_score=0
    best_model=None
    best_droite=None

    for k in range(N):
        score=0
        branche1=0
        branche2=0
        
        start=random.randint(0, pts.shape[0]//2)
        end=random.randint(pts.shape[0]//2, pts.shape[0]-1)
        middle=random.randint(start, end)
        
        droite1=[pts[start][:],pts[middle][:]]
        droite2=[pts[middle][:],pts[end][:]]
        
        model=np.array((pts[start],pts[end]))
        
        for P in pts:
            Dbr1=distanceDroite3D(droite1,P)
            Dbr2=distanceDroite3D(droite2,P)
            D=min(Dbr1,Dbr2)

            if Dbr1==D:
                branche1+=1
            else:
                branche2+=1


            if D<distanceMin:
                score+=1
                model=np.vstack((model,P))
                    
            firstLoop=False
        if score>pts.shape[0]//2 and score>best_score  :
            best_droite=[droite1,droite2]
            best_model=model
            best_score=score

        model=None
        
    if best_model is None:
        return None,None
    else:
        return best_model[2:],best_droite

In [12]:
def plotLines(lines,lines2=None,frame=None):

    for index, item in enumerate(lines): 
        if index == len(lines) -1:
            break
        cv2.line(frame, item[:2], lines[index + 1][:2], [10, 10, 255], 3)  
        
    for index, item in enumerate(lines2): 
        if index == len(lines2) -1:
            break
        cv2.line(frame, item[:2], lines2[index + 1][:2], [255, 10, 10], 2)  

def doRansac( position, frame,distMin=20,lastPosition=None,D3=False ):
    
    model,droite=RANSACcoude(position,N=position.shape[0]**3,distanceMin=distMin)

    
    if model is not None:
        cv2.line(frame, droite[0][1][:2], droite[0][0][:2], [0, 0, 0], 5) 
        cv2.line(frame, droite[1][1][:2], droite[1][0][:2], [0, 0, 0], 5) 
        #cv2.line(frame, droite[1][:2], droite[0][:2], [0, 0, 0], 5) 

        key = cv2.waitKey(0)
        

    return model,droite